In [1]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os

In [2]:
# Load and set environment variables
load_dotenv("mnemonic.env")
mnemonic=os.getenv("phrase")

In [3]:
type(mnemonic)

str

In [4]:
# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
from web3 import Web3
from bit import Key
from constants import *

SyntaxError: invalid syntax (<ipython-input-5-23698f0c3b06>, line 3)

In [6]:
# Import dependencies
import subprocess
import json
import os

from pprint import pprint # wacth out!
from dotenv import load_dotenv

# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")

# Import constants.py and necessary functions from bit and web3
from constants import BTC, ETH, BTCTEST

from bit import PrivateKeyTestnet
from bit.network import NetworkAPI

from web3 import Web3, middleware, Account
from web3.gas_strategies.time_based import medium_gas_price_strategy
from web3.middleware import geth_poa_middleware
 
# coonect to Web3
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))

# enable PoA middleware
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

mnemonic = os.getenv("phrase")

# Create a function called `derive_wallets`
def derive_wallets(coin=BTC, mnemonic=mnemonic, depth=3):
    command = f'php ./derive -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depth} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)

# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, to, amount):
    if coin == ETH:
        value = w3.toWei(amount, "ether")
        gasEstimate = w3.eth.estimateGas({"to":to, "from":account, "amount":value})
        return {
            "to":to,
            "from":account,
            "value":value,
            "gas":gasEstimate,
            "gasPrice": w3.eth.generateGasPrice(),
            "nonce":w3.eth.getTransactionCount(account),
            "chainID":w3.eth.chain_id
        }
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(to, amount, BTC)])

# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, to, amount):
    if coin == ETH:
        raw_tx = create_tx(coin, account.address, to, amount)
        signed = account.signTransactopn(raw_tx)
        return w3.eth.sendRawTransaction(signed.rawTransaction)
    if coin == BTCTEST:
        raw_tx = create_tx(coin, account, to, amount)
        signed = account.sign_transaction(raw_tx)
        return NetworkAPI.broadcast_tx_testnet(signed)
    
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(coin=ETH),
    BTC: derive_wallets(coin=BTCTEST)
}

In [7]:
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0xB33cd62caE1959255e83144900453FB3b73910F2',
   'xprv': 'xprvA3VA16B7aHGjVD8F23viNtvZmg9qcv5B9GvW5k2oJ687Nso673zEJqdrNSEQPAPzArgdLShUj28nWKVX3yB3qL5mVrCD5xdnZu3xre5RYku',
   'xpub': 'xpub6GUWQbi1Qeq2hhCi85Tik2sJKhzL2No2WVr6t8SQrRf6Fg8EebJUrdxLDirCQLsNAv6rVpeWdPYkqL9i4hA8pM7LRuTVp5jmoV6UhHoD4Sa',
   'privkey': '0x7ffbc915547e273aa371ffcbc9a606c8b00e518bb03fbe5fa998b46bf97e43c6',
   'pubkey': '0350f14cf14cc406390d745442411b17ff42b21673c3cb22ecbf2e7ebfa4603dfa',
   'pubkeyhash': '8716e16ba5de4d9c98b69dd80835cb26616f5c11',
   'index': 0},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x0a1ddeaDa85bADeD5A7561273D4C3b63D2D6F198',
   'xprv': 'xprvA3VA16B7aHGjZHKnCkDAj5fxqWSLJkb63EqNahL5mhLZ32ijUGoWCT9j6pjDuf2YnPLjG6sLdusgtjfbHZiRgzfFUXtwECmJcHkuBThheKa',
   'xpub': 'xpub6GUWQbi1Qeq2mmQFJmkB6DchPYGpiDJwQTkyP5jhL2sXuq3t1p7kkFUCx6JaBpeYHkJaDvkuwqD8EdghQA4J3D9ndM7vrGUriPeJWeHn2p9',
   'privkey': '0x093f843e542a63afb8ff875a2e0ce4f48551878a4f52d4